In [1]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub

In [3]:
model = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1").signatures["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
import random
import math

In [9]:
colorcodes = {}

In [23]:
def drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color):
    im_height, im_width, _  = image.shape
    left,top,right,bottom = int(xmin*im_width), int(ymin*im_height), int(xmax*im_width),int(ymax*im_height)
    cv2.rectangle(image,(left,top),(right,bottom),color = color,thickness = 4)
    FONT_SCALE = 5e-3
    THICKNESS_SCALE = 4e-3
    width = right-left
    height = bottom-top
    TEXT_Y_OFFSET_SCALE = 1e-2
    cv2.rectangle(
        image,
        (left,top- int(height * 6e-2)),
        (right,top),
        color = color,
        thickness = -1
        
    )
    cv2.putText(
        image,
        namewithscore,
        (left,top-int(height * TEXT_Y_OFFSET_SCALE)),
        fontFace = cv2.FONT_HERSHEY_PLAIN,
        fontScale = min(width,height)* FONT_SCALE,
        thickness = math.ceil(min(width,height)* THICKNESS_SCALE),
        color = (255,255,255)
    )

In [25]:
def draw(image,boxes,classnames,scores):
    boxesidx = tf.image.non_max_suppression(boxes,scores,max_output_size = 7, iou_threshold = 0.8,score_threshold = 0.1)
#     for i in range(len(boxes)):
    for i in boxesidx:
        ymin,xmin,ymax,xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0,255,30)
            c2 = random.randrange(0,255,25)
            c3 = random.randrange(0,255,50)
            colorcodes.update({classname: (c1,c2,c3)})
            color = colorcodes[classname]
        namewithscore = "{}:{}".format(classname,int(100*scores[i]))
        drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color)
        
    return image

In [27]:
image = cv2.imread("image2.jpg")
image = cv2.resize(image,(1000,900))
image2 = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
converted_img = tf.image.convert_image_dtype(image2,tf.float32)[tf.newaxis, ...]

In [29]:
detection = model(converted_img)

In [31]:
print(detection["detection_class_entities"])

tf.Tensor(
[b'Man' b'Clothing' b'Falcon' b'Owl' b'Falcon' b'Human arm' b'Human face'
 b'Eagle' b'Human arm' b'Human arm' b'Jeans' b'Clothing' b'Human arm'
 b'Person' b'Human arm' b'Human arm' b'Bird' b'Fashion accessory'
 b'Human face' b'Human hair' b'Bird' b'Person' b'Human arm' b'Flower'
 b'Footwear' b'Fashion accessory' b'Owl' b'Clothing' b'Human arm' b'Man'
 b'Human arm' b'Flower' b'Flower' b'Human arm' b'Person' b'Parrot'
 b'Flower' b'Eagle' b'Flower' b'Flower' b'Human arm' b'Fashion accessory'
 b'Footwear' b'Flower' b'Person' b'Clothing' b'Human eye' b'Flower'
 b'Human arm' b'Flower' b'Person' b'Human arm' b'Clothing' b'Human arm'
 b'Clothing' b'Human arm' b'Human arm' b'Human arm' b'Human face' b'Man'
 b'Human leg' b'Human arm' b'Plant' b'Flower' b'Human head' b'Chair'
 b'Flower' b'Person' b'Clothing' b'Window' b'Man' b'Human arm' b'Toy'
 b'Flower' b'Human leg' b'Human arm' b'Falcon' b'Human arm' b'Human arm'
 b'Chair' b'Window' b'Window' b'Shirt' b'Human leg' b'Person' b'Human 

In [33]:
result = {key:value.numpy() for key,value in detection.items()}
imagewithboxes = draw(image,result['detection_boxes'],result['detection_class_entities'],result["detection_scores"])
cv2.imshow("image",imagewithboxes)
cv2.waitKey(0)

-1